# Bayesian Additive Regression Trees

In this notebook, we will use the machine learning model of *Bayesian Additive Regression Tree* (BART)

BART is related to both random forests and boosting: each tree is constructed in a random manner (bagging and random forests) and each tree tries to capture signal not yet accounted for by the current model (boosting)

$1$ to $K$ regressive trees are fit in parallel and each tree is changed $B$ times as the sequential algorithm proceeds. At each step, the $K$ trees are changed in some way (*pertubrations*). At the nodes we can add or remove a branch or change the predicted values at each of th nodes. Basically we maintain $K$ trees and at each iteration we change each of those $K$ trees in some random way.

$\hat{f}^b_k(x)$ represents the prediction at $x$ for the $k$-th regression tree used in the $b$-th iteration. At the end of each iteration, the $K$ trees from that iteration will be summed: $\hat{f}^{b}(x) = \sum_{k=1}^{K}\hat{f}^b_k(x)$ for $b = 1, \dots, B$

For the *first iteration* of BART, all trees are initialized to have a single root node as the mean response values divded by the total number of trees: $\hat{f}^1_k(x) = \frac{1}{nK}\sum^{n}_{i=1}y_i$ thus $\hat{f}^1(x) = \frac{1}{n}\sum^{n}_{i=1}y_i$

After the first iteration: the $b$-th iteration, to update the $k$-th tree, we subtract from each response value the predictions from all **but** the $k$-th tree in order to obtain a *partial residual*:

$$r_i = y_i - \sum_{k'< k}\hat{f}^b_{k'}(x_i) - \sum_{k'>k}\hat{f}^{b-1}_{k'}(x_i)$$

Rather than fitting a fresh tree to this partial residual, BART randomly chooses a *pertubariton* to the tree from the previous iteration ($\hat{f}^{b-1}_{k}(x)$) from a set of possible pertubations, favoring ones that improve the fit to the partial residual. As mentioned before, there are 2 different ways for this pertubation to be made:  

1. Change the structure of the tree by adding or pruning branches
2. Change the prediction in each terminal node of the tree. (Basically the final predictions are changed a tiny bit). As in the tree could predict -0.5031 at a terminal node and a pertubation would make the prediction be -0.5110 instead.

To obtain a single prediction, we take the average after some $L$ *burn-in iterations*: $\hat{f}(x) = \frac{1}{B-L}\sum^B_{b = L+1}\hat{f}^b(x)$. Basically we ignore the first $L$ iterations. If $B=1000$ and $L=100$, we ignore the first $100$ steps and take the average of the remaining $900$ steps. During this period, the samples may not yet be representative of the target distribution because the chain is still "warming up" and finding its way to the regions of high probability density. We can also compute percenties of $f^{L+1}(x), \dots , f^B(x)$ which provides a measure of uncertainty of the final prediction

BART is a *Bayesian* algorithm because each time we randomly perturb a tree in order to fit the residuals, we are in fact draw a new tree from a *posterior* distribution. The BART algorithm can be viewed as a *Markov chain Monte Carlo* (MCMC) procedure for fitting the BART model.

Typically, large values are chosen for $B$ iterations and $K$ trees and a moderate value of $L$. For instance $K= 200$, $B = 1000$, $L=100$. 

A drawback of BART is that it is computationally expensive and can be slow

**SOURCE**: ISLP

In [15]:
import os 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from ISLP.bart import BART
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error

In [7]:
import sys
sys.executable

'C:\\Users\\Conno\\anaconda3\\python.exe'

In [18]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Importing Dataset

In [9]:
# Importing processed dataframe
path_project = "C:/Users/Conno/Documents/Career/Projects/Hospital_Charges/tree_based_models"

os.chdir(path_project)
plots_dir = 'bart_plots' # stores plots in plot folder

df = pd.read_csv("../df_processed.csv")

# do this later in data_cleaning file
bool_cols = df.select_dtypes(include=['bool']).columns
df[bool_cols] = df[bool_cols].astype(int)


## Train-Test Split

In [10]:
# same seed as every other mode
X = df.drop(columns = ["charges"])
y = df["charges"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 32)

In [ ]:
len(y_train)

In [ ]:
len(y_test)

# BART Model

In [27]:

# ndraws is B - L, the number of iterations after the burnin
B = 1000 # num iterations
L = 200
ndraws = B - L
K = 1000 # num trees

bart_model = BART(random_state = 32, burnin = L, num_trees = K, ndraw = ndraws, n_jobs = -1)
bart_model.fit(X_train, y_train)

BART(burnin=200, ndraw=800, num_trees=1000, random_state=32)

## Feature Importance

We will track feature importance by checking how many times each predictor (feature) appeared in the collection of trees

In [32]:
var_inclusion = pd.Series(bart_model.variable_inclusion_.mean(0),
                         index = X.columns).sort_values(ascending = False)
var_inclusion

hday                          43.34250
slos                          39.21875
dnrday                        38.58125
scoma                         38.57750
hospdead                      36.69500
dzgroup_chf                   36.18375
dzgroup_colon_cancer          36.03375
dzclass_cancer                35.74000
d_time                        35.64250
surv2m                        35.59250
dzgroup_lung_cancer           35.55500
sfdm2_adl>=4_>=5_if_sur_      35.43375
avtisst                       35.42875
dzgroup_copd                  34.61625
dzclass_copd_chf_cirrhosis    34.46500
num_co                        34.43500
surv6m                        34.38250
prg2m                         34.23250
dzclass_coma                  33.79500
dzgroup_mosf_w_malig          33.79500
sps                           33.66500
dzgroup_coma                  33.56500
aps                           33.47000
prg6m                         33.38875
dzgroup_cirrhosis             32.76875
sfdm2_no_m2_and_sip_pres_

The feature importance is similar to *bagging* and *random forest*, however `hday` was the most importance predictor meanwhile `slos` was the mort important for both *bagging* and *random forest*

## Predicting Test Set

In [28]:
y_pred_bart = bart_model.predict(X_test)

In [29]:
bart_mape = mean_absolute_percentage_error(y_test, y_pred_bart)
bart_mae = mean_absolute_error(y_test, y_pred_bart)
bart_mse = mean_squared_error(y_test, y_pred_bart)
bart_rmse = rmse(y_test, y_pred_bart)

print("BART MAPE:", round(bart_mape, 4))
print("BART MAE:", round(bart_mae, 4))
print("BART MSE:", round(bart_mse, 4))
print("BART RMSE:", round(bart_rmse, 4))

BART MAPE: 0.9437
BART MAE: 28367.4678
BART MSE: 3549067380.4923
BART RMSE: 59574.0496


Slightly better results than Bagging in terms of RMSE, however it took an extraodinarly long time despite having a good CPU (7800X3D). Bagging also won out in t